# Ablation Studies: GA-Optimized Decision Trees

This notebook systematically disables components to measure their individual contribution.

**Components tested:**
1. Mutation operators (threshold, feature, prune, expand)
2. Interpretability weight
3. Population size
4. Elitism ratio
5. Crossover probability

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

from ga_trees.fitness.calculator import FitnessCalculator, TreePredictor
from ga_trees.ga.engine import GAConfig, GAEngine, Mutation, TreeInitializer

X, y = load_iris(return_X_y=True)
n_features = X.shape[1]
n_classes = len(np.unique(y))
feature_ranges = {i: (float(X[:, i].min()), float(X[:, i].max())) for i in range(n_features)}
print(f'Dataset: {X.shape[0]} samples, {n_features} features, {n_classes} classes')

In [ ]:
def run_experiment(config_overrides=None, label='baseline', n_folds=5):
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    results = {'acc': [], 'nodes': [], 'depth': []}
    for train_idx, test_idx in skf.split(X, y):
        X_tr, X_te = X[train_idx], X[test_idx]
        y_tr, y_te = y[train_idx], y[test_idx]
        kwargs = dict(population_size=50, n_generations=30, random_state=42)
        if config_overrides:
            kwargs.update(config_overrides)
        config = GAConfig(**kwargs)
        init = TreeInitializer(n_features, n_classes, max_depth=5,
                               min_samples_split=10, min_samples_leaf=5)
        fc = FitnessCalculator(accuracy_weight=0.65, interpretability_weight=0.35)
        mut = Mutation(n_features, feature_ranges)
        engine = GAEngine(config, init, fc.calculate_fitness, mut)
        best = engine.evolve(X_tr, y_tr, verbose=False)
        pred = TreePredictor().predict(best, X_te)
        results['acc'].append(accuracy_score(y_te, pred))
        results['nodes'].append(best.get_num_nodes())
        results['depth'].append(best.get_depth())
    return {k: (np.mean(v), np.std(v)) for k, v in results.items()}

## 1. Mutation Operator Ablation

In [ ]:
mutation_configs = {
    'All operators (baseline)': None,
    'No threshold perturbation': {'mutation_types': {'threshold_perturbation': 0.0,
        'feature_replacement': 0.4, 'prune_subtree': 0.4, 'expand_leaf': 0.2}},
    'No prune_subtree': {'mutation_types': {'threshold_perturbation': 0.5,
        'feature_replacement': 0.35, 'prune_subtree': 0.0, 'expand_leaf': 0.15}},
    'No expand_leaf': {'mutation_types': {'threshold_perturbation': 0.5,
        'feature_replacement': 0.3, 'prune_subtree': 0.2, 'expand_leaf': 0.0}},
}

ablation_results = {}
for name, overrides in mutation_configs.items():
    print(f'Running: {name}...')
    ablation_results[name] = run_experiment(overrides, label=name)

df = pd.DataFrame({k: {m: f'{v[0]:.3f}±{v[1]:.3f}' for m, v in vals.items()}
                    for k, vals in ablation_results.items()}).T
df

## 2. Interpretability Weight Ablation

In [ ]:
# This tests what happens as we vary the accuracy/interpretability tradeoff
# We must modify the FitnessCalculator, not GAConfig, so we use a custom runner.
weight_results = {}
for iw in [0.0, 0.1, 0.2, 0.3, 0.5, 0.7]:
    label = f'interp_weight={iw}'
    print(f'Running: {label}...')
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    res = {'acc': [], 'nodes': []}
    for train_idx, test_idx in skf.split(X, y):
        X_tr, X_te = X[train_idx], X[test_idx]
        y_tr, y_te = y[train_idx], y[test_idx]
        config = GAConfig(population_size=50, n_generations=30, random_state=42)
        init = TreeInitializer(n_features, n_classes, max_depth=5,
                               min_samples_split=10, min_samples_leaf=5)
        fc = FitnessCalculator(accuracy_weight=1.0-iw, interpretability_weight=iw)
        mut = Mutation(n_features, feature_ranges)
        engine = GAEngine(config, init, fc.calculate_fitness, mut)
        best = engine.evolve(X_tr, y_tr, verbose=False)
        pred = TreePredictor().predict(best, X_te)
        res['acc'].append(accuracy_score(y_te, pred))
        res['nodes'].append(best.get_num_nodes())
    weight_results[label] = {k: (np.mean(v), np.std(v)) for k, v in res.items()}

df2 = pd.DataFrame({k: {m: f'{v[0]:.3f}±{v[1]:.3f}' for m, v in vals.items()}
                     for k, vals in weight_results.items()}).T
df2

## Summary

Compare all ablation results to determine which components contribute most.